In [12]:
import pandas as pd
data = pd.read_csv("merged_data.csv")
print("dataset is fetched")

dataset is fetched


In [13]:
import uuid
import csv
num_values =503422
guid_value =[str(uuid.uuid4()) for _ in range(num_values)]
csv_file = 'randomguid.csv'
column_header = 'ride_id'
with open (csv_file, 'w', newline='') as file:
    writer =csv.writer(file)
    writer.writerow([column_header])
    writer.writerows([value] for value in guid_value)

In [14]:
dfs =[]
randomguid =pd.read_csv("randomguid.csv")
file_name = ['randomguid.csv','merged_data.csv']
for  file_name in file_name:
    df = pd.read_csv(file_name)
    dfs.append(df)

merged_dataset = pd.concat(dfs, axis=1)

merged_dataset.to_csv('bike_search.csv', index = False)
Bikesearch = pd.read_csv('bike_search.csv')

In [15]:
print(Bikesearch.columns)

Index(['ride_id', 'ride_id.1', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')


In [21]:
from sklearn.impute import SimpleImputer

# Define imputation strategies
imputation_strategies = {
    'ride_id': 'drop',
    'ride_id.1': 'drop',
    'rideable_type': 'most_frequent',
    'started_at': 'most_frequent',
    'ended_at': 'most_frequent',
    'start_station_id': 'most_frequent',
    'end_station_id': 'most_frequent',
    'start_lat': 'mean',
    'start_lng': 'mean',
    'end_lat': 'mean',
    'end_lng': 'mean',
    'member_casual': 'most_frequent'
}

# Drop 'start_station_name' and 'end_station_name' columns
Bikesearch = Bikesearch.drop(columns=['start_station_name', 'end_station_name'])

# Identify columns with missing values
null_cols = Bikesearch.columns[Bikesearch.isnull().any()]

# Impute missing values
for col in null_cols:
    strategy = imputation_strategies.get(col, 'most_frequent')
    imputer = SimpleImputer(strategy=strategy)
    # Reshape the result of imputation to match the shape expected by DataFrame column
    imputed_values = imputer.fit_transform(Bikesearch[[col]]).ravel()
    # Assign the imputed values back to the DataFrame column
    Bikesearch[col] = imputed_values

# Save the DataFrame to a CSV file
Bikesearch.to_csv('bikecheck.csv', index=False)

# Read the CSV file back into a DataFrame for checking
checkdata = pd.read_csv("bikecheck.csv")
print("Null values count: \n", checkdata.isna().sum())

Null values count: 
 ride_id             0
ride_id.1           0
rideable_type       0
started_at          0
ended_at            0
start_station_id    0
end_station_id      0
start_lat           0
start_lng           0
end_lat             0
end_lng             0
member_casual       0
dtype: int64


In [22]:
# Create two separate DataFrames based on rideable type
classic_bike_data = Bikesearch[Bikesearch['rideable_type'] == 'classic_bike']
electric_bike_data = Bikesearch[Bikesearch['rideable_type'] == 'electric_bike']

# Save the DataFrames to separate CSV files
classic_bike_data.to_csv('classic_bike_data.csv', index=False)
electric_bike_data.to_csv('electric_bike_data.csv', index=False)

# Check the saved CSV files
print("Saved classic_bike_data.csv")
print("Saved electric_bike_data.csv")

Saved classic_bike_data.csv
Saved electric_bike_data.csv


In [37]:
print(classic_bike_data.columns)

Index(['ride_id', 'ride_id.1', 'rideable_type', 'started_at', 'ended_at',
       'start_station_id', 'end_station_id', 'start_lat', 'start_lng',
       'end_lat', 'end_lng', 'member_casual'],
      dtype='object')


In [38]:
print(electric_bike_data.columns)

Index(['ride_id', 'ride_id.1', 'rideable_type', 'started_at', 'ended_at',
       'start_station_id', 'end_station_id', 'start_lat', 'start_lng',
       'end_lat', 'end_lng', 'member_casual'],
      dtype='object')


In [79]:

import psycopg2

def create_classic_table(classic, table_name):
    try:
        cursor = classic.cursor()
        create_table_query = '''
        CREATE TABLE classic_bike_table (
    ride_id INTEGER,
    ride_id_1 INTEGER,
    rideable_type VARCHAR,
    started_at TIMESTAMP,
    ended_at TIMESTAMP,
    start_station_id INTEGER,
    end_station_id INTEGER,
    start_lat FLOAT,
    start_lng FLOAT,
    end_lat FLOAT,
    end_lng FLOAT,
    member_casual VARCHAR
);
        '''.format(table_name)
        cursor.execute(create_table_query)
        classic.commit()
        print("Table created successfully!")
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error while creating PostgreSQL table:", error)
    finally:
        if classic:
            cursor.close()
            classic.close()  # Close the database connection

# Replace these variables with your PostgreSQL connection details
host = "localhost"
database = "postgres"
user = "postgres"
password = "1234"  # Replace with your actual password

# Connect to the PostgreSQL database
try:
    classic = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )
    table_name = "classic_bike_table"  # Define the table name
    create_classic_table(classic, table_name)
except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)

Error while creating PostgreSQL table: relation "classic_bike_table" already exists



In [68]:
import psycopg2

def create_electric_table(electric, table_name):
    try:
        cursor = electric.cursor()
        create_table_query = '''
        CREATE TABLE electric_bike_table (
    ride_id VARCHAR(255),  -- Adjust the length as needed
    ride_id_1 VARCHAR(255),
    rideable_type VARCHAR(50),
    started_at TIMESTAMP,
    ended_at TIMESTAMP,
    start_station_id INTEGER,
    end_station_id INTEGER,
    start_lat NUMERIC,
    start_lng NUMERIC,
    end_lat NUMERIC,
    end_lng NUMERIC,
    member_casual VARCHAR(50)
);

        '''.format(table_name)
        cursor.execute(create_table_query)
        electric.commit()
        print("Table created successfully!")
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error while creating PostgreSQL table:", error)
    finally:
        if electric:
            cursor.close()
            electric.close()  # Close the database connection

# Replace these variables with your PostgreSQL connection details
host = "localhost"
database = "postgres"
user = "postgres"
password = "1234"  # Replace with your actual password

# Connect to the PostgreSQL database
try:
    electric = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )
    table_name = "electric_table"  # Define the table name
    create_electric_table(electric, table_name)
except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)



Table created successfully!


In [77]:
import pandas as pd
from sqlalchemy import create_engine

df = pd.DataFrame(classic_bike_data)

def load_dataframe_to_postgresql(dataframe, table_name, host, database, user, password):
    try:
        # Create SQLAlchemy engine
        engine = create_engine(f'postgresql://{user}:{password}@{host}/{database}')
        
        # Load DataFrame to PostgreSQL table
        dataframe.to_sql(table_name, engine, if_exists='append', index=False)
        
        print(f"Data loaded successfully into PostgreSQL table '{table_name}'!")
    except Exception as e:
        print("Error:", e)

# Define your PostgreSQL connection details
host = "localhost"
database = "postgres"
user = "postgres"
password = "1234" 
# Define the table name
table_name = "classic_bike_table"
# Call the function to load the DataFrame into PostgreSQL
load_dataframe_to_postgresql(df, table_name, host, database, user, password)


[SQL: INSERT INTO classic_bike_table (ride_id, "ride_id.1", rideable_type, started_at, ended_at, start_station_id, end_station_id, start_lat, start_lng, end_lat, end_lng, member_casual) VALUES (%(ride_id__0)s, %(ride_id_1__0)s, %(rideable_type__0)s, %(star ... 254515 characters truncated ...  %(start_lat__999)s, %(start_lng__999)s, %(end_lat__999)s, %(end_lng__999)s, %(member_casual__999)s)]


In [71]:
import pandas as pd
from sqlalchemy import create_engine

# Define your DataFrame with your data
df = pd.DataFrame(electric_bike_data)

def load_dataframe_to_postgresql(dataframe, table_name, host, database, user, password):
    try:
        # Create SQLAlchemy engine
        engine = create_engine(f'postgresql://{user}:{password}@{host}/{database}')
        
        # Load DataFrame to PostgreSQL table
        dataframe.to_sql(table_name, engine, if_exists='append', index=False)
        
        print(f"Data loaded successfully into PostgreSQL table '{table_name}'!")
    except Exception as e:
        print("Error:", e)


# Define your PostgreSQL connection details
host = "localhost"
database = "postgres"
user = "postgres"
password = "1234" 
# Define the table name
table_name = "electric_bike_table"
# Call the function to load the DataFrame into PostgreSQL
load_dataframe_to_postgresql(df, table_name, host, database, user, password)
print(electric_bike_data)

[SQL: INSERT INTO electric_bike_table (ride_id, "ride_id.1", rideable_type, started_at, ended_at, start_station_id, end_station_id, start_lat, start_lng, end_lat, end_lng, member_casual) VALUES (%(ride_id__0)s, %(ride_id_1__0)s, %(rideable_type__0)s, %(sta ... 254516 characters truncated ...  %(start_lat__999)s, %(start_lng__999)s, %(end_lat__999)s, %(end_lng__999)s, %(member_casual__999)s)]
                                     ride_id         ride_id.1  rideable_type  \
0       d98d3ae6-ae41-4a2a-8535-b52c9383cc30  C2F7DD78E82EC875  electric_bike   
1       d692fca5-d1ff-4b4a-b1a3-d0eb2627f1b0  A6CF8980A652D272  electric_bike   
8       6997d20f-bab4-45a9-a5ae-3ffd22f64e9d  154222B86A338ABD  electric_bike   
11      edc17ae0-48f8-4f56-8c26-4cb08e64c1f3  006D3F363CF9600B  electric_bike   
14      a7f9deeb-d9b0-4442-8c5e-4f8a02098763  E119D4E6BF686601  electric_bike   
...                                      ...               ...            ...   
503406  5ffda94e-bbc6-47f2-a660-778618